In [4]:
from pgml import *
from AKV import *
import networkx as nx
import matplotlib.pyplot as plt

In [5]:
def draw_graph(model: Model):
    DG = nx.DiGraph()
    world_size = model.worlds_size
    DG.add_nodes_from(list(range(model.worlds_size)))
    DG.add_weighted_edges_from(
        [
            (i, j, model.relation[i][j])
            for i in range(world_size)
            for j in range(world_size)
            if model.relation[i][j] != 0
        ]
    )
    # labels = nx.get_edge_attributes(G, "weights")
    nx.draw(DG)

In [6]:
# Faintly connected graph...
NUM_AGENTS = 10
WEAK_INFLUENCE = 0.1
STRONG_INFLUENCE = 0.5
influence_graph = InfluenceGraphs.faintly(10, WEAK_INFLUENCE, STRONG_INFLUENCE)

# Not necessary for faintly connected graph, but it is necessary for any non symetric graph.
relation = [
    [influence_graph[j][i] for j in range(NUM_AGENTS)] for i in range(NUM_AGENTS)
]

# Remove self-loops.
for i in range(10):
    relation[i][i] = 0

# ...with mildly initial state.
belief_array = [InitialConfigurations.mildly(NUM_AGENTS)]
belief_array.append([1 - belief_array[0][i] for i in range(NUM_AGENTS)])

# # Valuation 1 is the same as the belief_array
# valuation1 = {"p": belief_array[0]}

# # ...and Valuation 2 is the inverse of valuation 1 for this example.
# valuation2 = {"p": belief_array[1]}

# # AKV model
# akv_model = AKV(belief_array, influence_graph)

# # PGML model
# pgml_model = Model(NUM_AGENTS, relation, valuation1, valuation2)

# draw_graph(pgml_model)

In [7]:
formula = Box(Variable("p"))
akv_model = AKV(belief_array, influence_graph)

# Run AKV 10 times
NUM_STEPS = 10
for i in range(NUM_STEPS):
    akv_model.overall_class_update()

# Get polarization array
polarization = akv_model.get_polarization(k=11)

# Create a PGML model for each step
pgml_models = []
for i in range(NUM_STEPS + 1):
    pgml_models += Model(
        NUM_AGENTS,
        relation,
        {"p": akv_model.belief_array[i][0]},
        {"p": akv_model.belief_array[i][1]},
    )

# Build table for agent
AGENT = 0
table = [
    {
        "polarization": polarization[i],
        "agent valuation1": akv_model.belief_array[i][0],
        "agent valuation2": akv_model.belief_array[i][1],
        "pgml valuation": formula.valuation1(pgml_models[i], AGENT),
    }
    for i in range(NUM_STEPS + 1)
]

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)